In [11]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from collections import defaultdict
import random
import csv

In [12]:
# Define the parameters
data_path = "./cost_10_07.csv"
part_col = "Part"
dep_arr_site_col = "Departure/Arrival site"
arr_dep_site_col =  "Arrival/Departure site"
cost_col = "Cost"
PBS = [(2, 1), (3, 1), (4, 1), (5, 2) , (6, 2), (7, 2), (8, 3), (9, 4), (10, 4)]
subpart = [(2,3),(2,4),(3,4),(5,6),(5,7),(6,7),(9,10)]
lambda_1 = 100
lambda_2 = 100
lambda_3 = 100

In [13]:
# Define the function to create the cost matrix
def cost_matrix_df(data_path, part_col, dep_arr_site_col, arr_dep_site_col, cost_col):
    # Load the data and create a DataFrame
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)

    # Create the pivot DataFrame
    part_range = range(2, data_df[part_col].max() + 1)  # Start from 2
    site_range = range(1, max(data_df[dep_arr_site_col].max(), data_df[arr_dep_site_col].max()) + 1)

    pivot_df = (
        pd.DataFrame(
            index=pd.MultiIndex.from_product(
                [part_range, site_range, site_range], 
                names=['Part', 'Departure_Site', 'Arrival_Site']
            )
        )
        .reset_index()
        .merge(data_df.assign(Part_Column=data_df[part_col]), how='left', left_on=['Part', 'Departure_Site', 'Arrival_Site'], right_on=[part_col, dep_arr_site_col, arr_dep_site_col])
        .assign(Cost=lambda df: df[cost_col].fillna(0))
        .pivot_table(index=['Part', 'Departure_Site'], columns=['Part_Column', 'Arrival_Site'], values='Cost', fill_value=0)
        .pipe(lambda df: df.reorder_levels(['Part_Column', 'Arrival_Site'], axis=1))
        .pipe(lambda df: df.reindex(columns=pd.MultiIndex.from_product([part_range, site_range], names=df.columns.names), fill_value=0))
        .pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([part_range, site_range], names=df.index.names), fill_value=0))
        .pipe(lambda df: df + df.T)  # Make the DataFrame symmetric
    )

    return pivot_df

In [14]:
# Define the function to create the cost function
def cost_function_qubo(PBS):
    # Get the range of parts and sites
    pivot_df = cost_matrix_df(data_path, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)
    site_range = pivot_df.index.get_level_values('Departure_Site').unique()

    # Convert the pivot DataFrame to a QUBO dictionary
    cost_function = {((r, i), (s, j)): pivot_df.loc[(r, i), (r, j)] 
            for r, s in PBS for i in site_range for j in site_range if i != j and r != s and (r, i) in pivot_df.index and (r, j) in pivot_df.index}

    return cost_function

In [15]:
# Define the function to create the first constraint function
def penalty_C1_qubo(PBS, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()

    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C1 = {}
    for r, _ in PBS:
        for i in range(1, max_sites + 1):
            penalty_qubo_C1[((r, i), (r, i))] = -penalty
            for j in range(i + 1, max_sites + 1):
                penalty_qubo_C1[((r, i), (r, j))] = 2 * penalty
                penalty_qubo_C1[((r, j), (r, i))] = 2* penalty

    return penalty_qubo_C1

In [16]:
# Define the function to create the second constraint function
def penalty_C2_qubo(PBS, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()

    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C2 = {}
    for r, s in PBS:
        for i in range(1, max_sites + 1):
            # Add a penalty if the sum of Xr,iXs,i is non-zero
            penalty_qubo_C2[((r, i), (s, i))] = penalty

    return penalty_qubo_C2

In [17]:
# Define the function to create the third constraint function
def penalty_C3_qubo(subpart, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()


    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C3 = {}
    for r, s in subpart:
        if r < s:
            for i in range(1, max_sites + 1):
                # Add a penalty if the origins of two subparts of a common part are the same
                penalty_qubo_C3[((r, i), (s, i))] = penalty

    return penalty_qubo_C3

In [18]:
# Auxiliary function to multiply penalties by QUBO dictionaries

def multiply_penalty_qubo(penalty, qubo_dict):
    # Create a new QUBO dictionary to hold the result
    result_qubo = {}

    # Multiply each value in the QUBO dictionary by the penalty
    for key, value in qubo_dict.items():
        result_qubo[key] = penalty * value

    return result_qubo

In [19]:
# Auxiliary function to add QUBO dictionaries
def add_qubo_dicts(qubo_dict1, qubo_dict2):
    # Create a new defaultdict to hold the result
    result_qubo = defaultdict(int)

    # Add the values from the first QUBO dictionary
    for key, value in qubo_dict1.items():
        result_qubo[key] += value

    # Add the values from the second QUBO dictionary
    for key, value in qubo_dict2.items():
        result_qubo[key] += value

    return dict(result_qubo)

In [20]:
C = cost_function_qubo(PBS)
C1 = penalty_C1_qubo(PBS)
C2 = penalty_C2_qubo(PBS)
C3 = penalty_C3_qubo(subpart)

In [21]:
# List of tuples (penalty, QUBO dictionary)
penalties_qubos = [(1, C), (lambda_1, C1), (lambda_2, C2), (lambda_3, C3)]

# Initialize Q as an empty dictionary
Q = {}

# Iterate over the list of tuples
for penalty, qubo in penalties_qubos:
    # Multiply the QUBO dictionary by its penalty
    qubo_multiplied = multiply_penalty_qubo(penalty, qubo)
    
    # Add the multiplied QUBO dictionary to Q
    Q = add_qubo_dicts(Q, qubo_multiplied)

In [23]:
len(cost_function_qubo(PBS))

378